##### 参数种类：定位参数和关键字参数传入的形参、定位参数元组、关键字参数字典、仅限关键字参数、仅限定位参数

In [17]:
from inspect import signature
def func(a,b=2,*arg,**kwarg):
    print(a,b,arg,kwarg)

In [18]:
sig = signature(func)
for name, param in sig.parameters.items():
    print(param.kind, name, param.default)

POSITIONAL_OR_KEYWORD a <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD b 2
VAR_POSITIONAL arg <class 'inspect._empty'>
VAR_KEYWORD kwarg <class 'inspect._empty'>


##### 支持函数式编程的包---operator模块、functools模块

In [19]:
'''itemgetter 使用 [] 运算符，因此它不仅支持序列，
还支持映射和任何实现 __getitem__ 方法的类
attrgetter 提取名称提取对象的属性 相当于 ’.‘ '''
from operator import itemgetter
metro_data = [ ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
 ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),]
sorted(metro_data,key= itemgetter(1))

[('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333))]

In [20]:
dic={1:2,3:4}
f = itemgetter(1)
f(dic)

2

In [21]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long') #
Metropolis = namedtuple('Metropolis', 'name cc pop coord') 
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) 
                   for name, cc, pop, (lat, long) in metro_data]
from operator import attrgetter
sorted(metro_areas, key = attrgetter('coord.lat'))
f = attrgetter('name') 
f(metro_areas[0]) # 相当于Metropolis.name

'Tokyo'

In [22]:
from operator import methodcaller
# methodcaller 创建的函数会在对象上调用参数指定的方法
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [23]:
# 使用functools.partial 冻结参数
from operator import mul
from functools import partial,reduce
# partial 对象提供了访问原函数和固定参数的属性
#partialmethod 函数用于处理方法
def fun(a,b,c):
    return a*b*c
triple = partial(fun,3)
reduce(triple,range(1,5))

648

#### 装饰器
装饰器：可调用对象，其参数时另一个函数装饰器可能会处理被装饰的函数，然后把它返回，或者将其替换成另一个函数或可调用对象。

闭包：保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用，但是仍能使用那些绑定。只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量。
<code>
    @clock
    def factorial(n):
等价于factorial = clock(factorial)</code>

In [1]:
'''如何让装饰器函数接受其他参数？ 
创建一个装饰器工程函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上。'''
import json
# 在定义装饰器后真正的参数只有一个参数，那就是被装饰函数的指针。
def json_output(indent = None, sort_keys = False):
    def actual_decorator(decorated):
        def inner(*args,**kwargs):
            result = decorated(*args,**kwargs)
            return json.dumps(result,indent=indent,sort_keys = sort_keys)
        return inner
    return actual_decorator
@json_output(indent = 8) # 函数的返回值作为装饰器
def do_nothing():
    return {'status':'done','func':'yes'}
do_nothing()

running register(active=True)->decorate(<function f1 at 0x0000029E672B79D8>)


In [ ]:
import functools
def log(func):
    @functools.wraps(func) #原始函数的__name__等属性复制到wrapper()函数中
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper
@log
def now():
    print('2015-3-25')

#### Python 优雅地dumps非标准类型

In [ ]:
import json
from datetime import datetime
from decimal import Decimal
from functools import singledispatch

class MyClass:
    def __init__(self, value):
        self._value = value
    def get_value(self):
        return self._value

mc = MyClass('myclass')
dm = Decimal('11.11')
dt = datetime.now()
@singledispatch
def convert(o):
    raise TypeError('can not convert type')
@convert.register(datetime)
def _(o):
    return o.strftime('%b %d %Y %H: %M: %S')
@convert.register(MyClass)
def _(o):
    return o.get_value()
@convert.register(Decimal)
def _(o):
    return float(o)
class ExtendJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        try:
            return convert(obj)
        except TypeError:
            return super(ExtendJSONEncoder, self).default(obj)
data = {
    'mc':mc, 'dm': dm, 'dt': dt
}
json.dumps(data, cls = ExtendJSONEncoder)

In [ ]:
#创建一个自己的迭代辅助函数
def with_next(iterable):
    iterator = iter(iterable)
    current = next(iterator)
    for next_item in iterator:
        yield next_item,current
        current = next_item

difference = []
for next_item,current in with_next(events):
    difference.append(next_item-current)
difference

In [ ]:
'''把变量从内存中变成可存储或传输的过程称之为序列化，在Python中叫pickling.
序列化之后，就可以把序列化后的内容写入磁盘，或者通过网络传输到别的机器上。
反过来，把变量内容从序列化的对象重新读到内存里称之为反序列化，即unpickling。'''
'''数据库表是一个二维表，包含多行多列。把一个表的内容用Python的数据结构表示出来的话，
可以用一个list表示多行，list的每一个元素是tuple，表示一行记录。
但是用tuple表示一行很难看出表的结构。如果把一个tuple用class实例来表示，
就可以更容易地看出表的结构来。
这就是传说中的ORM技术：Object-Relational Mapping，把关系数据库的表结构映射到对象上。'''

In [ ]:
class A:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def mydefault(self,*args):
        print('default',str(args[0]))
# 只有当没有定义的方法调用时，才会有调用方法__getattr__。
    def __getattr__(self,name):
        return self.mydefault
    
a = A(1,2)
a.first(5)

In [ ]:
# 可切片序列
from array import array
import reprlib
import math
import numbers
class Vector:
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components) 
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __getitem__(self,index):
        cls = type(self)
        if isinstance(index,slice):
            return cls(self._components[index])
        elif isinstance(index,numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [ ]:
class Seq:
    def __init__(self,component):
        self._components = list(component)
    def __repr__(self):
        components = reprlib.repr(self._components) 
        # 使用reprlib.repr() 获取self._components的有限长度表示形式。
        components = components[components.find('['):] 
        # 把字符串插入 Vector 的构造方法调用之前，去掉前面的array('d' 和后面的 )
        return 'Seq({})'.format(components)
    def __getitem__(self,index):
        cls = type(self)
        if isinstance(index,tuple):
            return cls(self._components[index[0]][index[1]])
        else:
            raise TypeError()

In [ ]:
# 动态存取属性
from array import array
import reprlib
import math
import numbers
import functools
import operator
class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        self._components = array(self.typecode, components) 
    def __getattr__(self,name):
        cls = type(self)
        if len(name)==1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos <len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}' 
        raise AttributeError(msg.format(cls, name))
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1: 
            if name in cls.shortcut_names: 
                error = 'readonly attribute {attr_name!r}'
            elif name.islower(): 
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = '' 
        if error: 
            msg = error.format(cls_name=cls.__name__, attr_name=name)
            raise AttributeError(msg)
        super().__setattr__(name, value)

'''多数时候，
如果实现了 __getattr__ 方法，那么也要定义__setattr__ 方法，以防对象的行为不一致。'''


In [ ]:
# 在迭代操作和其他操作的时候，怎样只保留最后N个元素
from collections import deque #双向队列

def search(lines, pattern, history = 5):
    previous_line = deque(maxlen= history)
    for line in lines:
        if pattern in line:
            yield line, previous_line
        previous_line.append(line)
with open('py.txt','r') as f:
    for line, previous_line in search(f,'how',3):
        for pline in previous_line:
            print(pline, end=' ')
        print(line, end=' ')
        print('-'*20)
    

In [ ]:
# 查找最大或最小的n个元素
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
display(heapq.nlargest(3,nums), heapq.nsmallest(3,nums))
portfolio = [ {'name': 'IBM', 'shares': 100, 'price': 91.1},
             {'name': 'AAPL', 'shares': 50, 'price': 543.22},
             {'name': 'FB', 'shares': 200, 'price': 21.09}, 
             {'name': 'HPQ', 'shares': 35, 'price': 31.75}, 
             {'name': 'YHOO', 'shares': 45, 'price': 16.35}, 
             {'name': 'ACME', 'shares': 75, 'price': 115.65} ] 
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price']) 
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])
print(expensive)

In [ ]:
class PriorityQueue:
    def __init__(self):
        self.queue = []
        self.index = 0
    def push(self, item, priority):
        heapq.heappush(self.queue, (-priority, self.index, item))
        self.index+=1 # index 变量也在相 同优先级元素比较的时候起到重要作用
        
    def pop(self):
        return heapq.heappop(self.queue)[-1]
    
class Item:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return 'Item({!r})'.format(self.name)
q= PriorityQueue()
q.push(Item('foo'), 1) 
q.push(Item('bar'), 5) 
q.push(Item('spam'), 4) 
q.push(Item('grok'), 1) 

In [ ]:
# 删除序列相同的元素同时保持顺序不变
#序列上的值都是 hashable 类型
def dedupe(items): 
    seen = set() 
    for item in items: 
        if item not in seen: 
            yield item 
            seen.add(item)
a = [1, 5, 2, 1, 9, 1, 5, 10]
list(dedupe(a))
#想消除元素不可哈 希 (比如 dict 类型) 的序列中重复元素
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen: 
            yield item 
            seen.add(val) 
a = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}] 
list(dedupe(a, key=lambda d: (d['x'],d['y'])))

In [ ]:
# 通过某个字段将记录分组
from itertools import groupby
rows = [ {'address': '5412 N CLARK', 'date': '07/01/2012'}, 
        {'address': '5148 N CLARK', 'date': '07/04/2012'}, 
        {'address': '5800 E 58TH', 'date': '07/02/2012'}, 
        {'address': '2122 N CLARK', 'date': '07/03/2012'}, 
        {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
        {'address': '1060 W ADDISON', 'date': '07/02/2012'}, 
        {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
        {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}]
rows.sort(key=itemgetter('date')) #先排序，后分组
for date, items in groupby(rows, key=itemgetter('date')): 
    print(date) 
    for i in items:
        print(' ', i)
#要根据指定的字段将数据排序。因为 groupby() 仅仅检查连续的元素


In [ ]:
### 字符串搜索
import re
text = 'Today is 11/27/2012. PyCon starts 3/13/2013' 
datapa = re.compile(r'(\d+)/(\d+)/(\d+)')
itertext = datapa.finditer(text)
for tex in itertext:
    print('{2}-{0}-{1}'.format(*tex.groups()))

datepat = re.compile(r'(\d+)/(\d+)/(\d+)') 
datepat.search('ddd11/27/2012').groups()

### 字符串替代

 re.sub(r'(\d+)/(\d+)/(\d+)', r'\3/\1/\2', text) 
import re
s = '2016-05-23  2016-05-24 2016-05-25'
re.sub(r'(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})',
       r'\g<month>/\g<day>/\g<year>',s)
# re.sub("(\d{4})-(\d{2})-(\d{2})",r"\2/\3/\1",s) \2 表示第个group

from calendar import month_abbr 
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3)) 
datepat.sub(change_date, text) 

newtext,n=datepat.subn(r'\3-\1-\2',text)
display(newtext,n)# n表示替代发生次数

### 字符串忽略大小写的搜索和替代

text = 'UPPER PYTHON, lower python, Mixed Python' 
find= re.findall('python', text, flags=re.IGNORECASE) 
sub = re.sub('python', 'snake', text, flags=re.IGNORECASE)
display(find,sub)

def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace
re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE) 
#group 是所有满足匹配的字符串的字符串。

In [ ]:
#在一行中调用多个函数。
def add(a, b):
    return a + b
def subtract(a, b):
    return a - b
a, b = 4, 5
print((subtract if a > b else add)(a, b)) # 9

#合并两个词典。
def merge_dictionaries(a, b)
   return {**a, **b}
a = {  x : 1,  y : 2}
b = {  y : 3,  z : 4}
#返回列表中出现的最常见元素。
def most_frequent(list):
    return max(set(list), key = list.count)
  
list = [1,2,1,2,3,2,1,4,2]
most_frequent(list)
#输出一个执行单元中的所有结果
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
#如果要恢复成初始设定：
InteractiveShell.ast_node_interactivity = "last_expr"

import itertools

for line in itertools.dropwhile(lambda line: line.startswith("//"), 
                                string_from_file.split("\n")):
    print(line)

In [ ]:
'''迭代器切片: 如果对迭代器进行切片操作，会返回一个「TypeError」，
提示生成器对象没有下标，但是我们可以用一个简单的方案来解决这个问题：'''
#先实现迭代器对象，然后再实现可迭代对象。
# islice(iterable, stop) --> islice object
# islice(iterable, start, stop[, step]) --> islice object
### 可迭代对象和迭代器
import itertools
s = itertools.islice((i*2 for i in range(30)), 10, 20)  
for val in s:
    print(val)

In [ ]:
# 如何创建可管理的对象属性
class Attr:
    def __init__(self, name, type_):
        self.name = name
        self.type_ = type_
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('not %s',self.type_)
        instance.__dict__[self.name]= float(value)
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]
        
class A:
    R = Attr('R',(int,float))
    def __init__(self, radius):
        self.R = radius
a = A(5)
a.R = 6
a.R

class Circle2:
    def __init__(self, radius):
        self.radius = radius
    @property
    def R(self):
        return self.radius 
    @R.setter
    def R(self, value):
        if isinstance(value, (int, float)):
            self.radius = float(value)
        else:
            raise ValueError('Wrong')
            
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    return property(qty_getter, qty_setter)
class LineItem:
    #使用工厂函数把自定义的特性weight定义为类属性。
    weight = quantity('weight')
    #构建另一个自定义的特性，price。
    price = quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

In [ ]:
# 如何派生内置不可变类型并修改其实例化行为
class IntTuple(tuple):
    def __new__(cls, iterable): # 调用__init__函数之前，会先调用__new__函数
        g= (x for x in iterable if isinstance(x, int) and x>0)
        return super(IntTuple, cls).__new__(cls, g)# 继承父类的__new__函数
    def __init__(self,iterable):
        print(self)
        super(IntTuple, self).__init__( )
t =  IntTuple([1,-1,'abc',6,['x','y'],3])